In [1]:
import json
from transformers import BertTokenizerFast, BertForTokenClassification
import regex as re
import torch
import torch.nn as nn
from torch.nn import functional as F
from transformers import BertModel
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm
import random

C:\Users\Emil\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [3]:
bert = "KB/bert-base-swedish-cased" #Options: KB/bert-base-swedish-cased, bert-base-uncased
reg = r'\p{L}+|-|\*'
max_length = 100
batch_size = 16
lr = 3e-5
epochs = 10

In [5]:
with open("train_data_our.json", encoding = "utf-8") as f:
    data = json.load(f)
    f.close()

print(len(data))
split = (int) (len(data) * 0.8)
train_data = data[:split]
test_data = data[split:-1]

1011


In [5]:
tokenizer = BertTokenizerFast.from_pretrained(bert)
inputs = []
labels = []


regtest = r'\p{L}+|[\p{P}]|[\p{N}]+|[^\p{L}\s]+'


for data in train_data:
    inputs.append(tokenizer(re.findall(regtest,data[0]),return_tensors="pt",padding = 'max_length',max_length=max_length,truncation=True,is_split_into_words=True))

    
for i in range(len(inputs)):
    hword = tokenizer(re.findall(regtest,train_data[i][1]),return_tensors="pt",padding = 'max_length',max_length=20,truncation=True,is_split_into_words=True)
    hword = hword['input_ids'].tolist()[0] 
    hword = hword[1:hword.index(3)]
    words = inputs[i]['input_ids'].tolist()[0]

    
    start = -1
    end = -1

    for j in range(len(words)):
        if words[j:j+len(hword)] == hword:
            start = j
            end = j + len(hword)
            break;

    label = [0] * max_length
    label[start:end] = [1] * (end-start)
    
    labels.append(torch.tensor(label))

input_ids = []
attention_masks = []

for input_id in inputs:
    input_ids.append(input_id['input_ids'].squeeze(0))
    attention_masks.append(input_id['attention_mask'].squeeze(0))
for i in range(len(input_ids)):
    print(tokenizer.convert_ids_to_tokens(inputs[i]['input_ids'][0]))
    print(labels[i])
    print(train_data[i][1])


input_ids = torch.stack(input_ids)
labels = torch.stack(labels)
attention_masks = torch.stack(attention_masks)

print(input_ids.size())
print(labels.size())
print(attention_masks.size())


['[CLS]', 'A', '.', '[UNK]', '.', 'vul', '##g', '.', ',', 'förkortning', 'för', 'Ann', '##o', '[UNK]', 'vul', '##gar', '##is', '(', 'lat', '.', ')', ',', 'året', 'efter', 'den', 'vanliga', 'tid', '##räkningen', '.', '[UNK]', '[UNK]', '«', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
tensor([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [6]:
dataset = TensorDataset(input_ids,labels,attention_masks)
dataloader = DataLoader(dataset, batch_size = batch_size,shuffle = True)

In [7]:
model = BertForTokenClassification.from_pretrained(bert, num_labels=2).to(device) #KB/bert-base-swedish-cased bert-base-uncased
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at KB/bert-base-swedish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [82]:
for epoch in range(epochs):
    
    model.train()
    total_loss = 0

    for inputs,target,attention_mask in tqdm(dataloader):

        inputs = inputs.to(device)
        target = target.to(device)
        attention_mask = attention_mask.to(device)

        #Forward
        outputs = model(input_ids=inputs,labels=target,attention_mask = attention_mask)

        loss = outputs.loss
        logits = outputs.logits

        #Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

100%|██████████| 76/76 [00:05<00:00, 13.57it/s]


Epoch 1, Loss: 0.0511


100%|██████████| 76/76 [00:05<00:00, 14.45it/s]


Epoch 2, Loss: 0.0034


100%|██████████| 76/76 [00:05<00:00, 14.50it/s]


...

Epoch 8, Loss: 0.0001


100%|██████████| 76/76 [00:05<00:00, 14.44it/s]


Epoch 9, Loss: 0.0001


100%|██████████| 76/76 [00:05<00:00, 14.32it/s]

Epoch 10, Loss: 0.0001


In [83]:
torch.save(model.state_dict(), "annotatorActive.pt") # ALDRIG DEN HÄRRRRR

In [8]:
model = BertForTokenClassification.from_pretrained(bert, num_labels=2).to(device)
model.load_state_dict(torch.load("annotatorActive.pt",weights_only = True))
model.eval()

Some weights of BertForTokenClassification were not initialized from the model checkpoint at KB/bert-base-swedish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50325, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [9]:
#Validation
tp = 0
tf = 0
fp = 0
fn = 0
c= 0

extractlist = []

for desc, headword in tqdm(test_data):
    last = -1
    converter = tokenizer(re.findall(regtest,desc),return_tensors="pt",padding = 'max_length',max_length=max_length,truncation=True,is_split_into_words=True)
    input_ids = converter["input_ids"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids = input_ids, attention_mask = converter['attention_mask'].to(device))

    probs = F.softmax(outputs.logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)
    predlist = preds.tolist()
    i = 0
    word = ""

    while i < len(converter.word_ids()):
        
        if predlist[0][i] == 1:
            if converter.word_ids()[i] != last and converter.word_ids()[i] != None:
                word = word + " " + re.findall(regtest,desc)[converter.word_ids()[i]]
            
            last = converter.word_ids()[i]
    
        i +=1


    word = re.findall(reg,word)
    extractlist.append([word,desc])
    
    headword = headword.replace(" ","")
    wordtemp = ""
    if len(word)>0:
        for w in word:
            wordtemp += w
            
    word = wordtemp

    
    if word == headword:
        tp += 1
    elif len(word) > 0 and len(headword) == 0:
        fp += 1
    elif len(word) == 0 and len(headword) > 0:
        print("False Negative")
        print(word)
        print(headword)
        print(desc)
        fn += 1
    else:
        tf += 1

print("True Positive: ", tp/len(test_data))
print("True Negative: ", tf/len(test_data))
print("False Positive: ", fp/len(test_data))
print("False Negative: ", fn/len(test_data))
precision = tp/(tp+fp)
recall = (tp)/(tp +fn)
print("Precision: ", tp/(tp+fp),", ","Recall: ",(tp)/(tp +fn),", ","Accuracy: ",(tp +tf)/len(test_data),", ","F1: ", 2* ((precision * recall)/(precision+recall)))

 29%|██▉       | 89/302 [00:03<00:03, 65.99it/s]

False Negative

Norskkonst
Norsk konst framträder under den äldsta kristna tiden
hufvudsakligen som arkitektur. Stenbyggnadskonst
infördes af angelsaxiska och iriska präster
och munkar. De äldsta stenkyrkorna voro ganska
små, rektangulära, enskeppiga. Ett lägre och
smalare kor med rätlinig afslutning fogades
ibland till kyrkan? skepp, och denna täcktes med
ett spärr-tak. Denna irisk-angelsaziska period
aflöses under senare hälften af 1000-talet af den
anglonorman-diska perioden. De större kyrkorna byggdes
nu oftast treskeppiga med halfcirkelformigt eller
linje-rät korafslutning samt hvälfda. Bland kyrkor
från denna tid märkas Gamle Åkers kyrka i Kristiania,
kyrkor i Gran, Ringsaker, ruinen af Hamars domkyrka,
Mariakyrkan i Bergen, skeppet i Stavangers domkyrka,
kyrkor i Hove, Ysernes, Msere samt tvärhuset i
Trondhjems domkyrka. Från detta skede kvarstå ruiner
af flera kloster, bl. a. på Hovedöen vid Oslo. Vid
slutet af 1100-talet börja öfvergångs-former till
gotisk stil göra sig gällan

100%|██████████| 302/302 [00:06<00:00, 48.36it/s] 

True Positive:  0.9271523178807947
True Negative:  0.059602649006622516
False Positive:  0.006622516556291391
False Negative:  0.006622516556291391
Precision:  0.9929078014184397 ,  Recall:  0.9929078014184397 ,  Accuracy:  0.9867549668874173 ,  F1:  0.9929078014184397


In [18]:
#Predict
editionV = "U3_E."
v = 1

with open("TREDJE_PARAGRAPHS.json", encoding = "utf-8") as f:
    edition = json.load(f)
    f.close()

edition2 = []
for i in range (len(edition)):
    edition2.append(edition[i]['paragraph'])

extractlist = []


for desc in tqdm(edition2):
    
    converter = tokenizer(re.findall(regtest,desc),return_tensors="pt",padding = 'max_length',max_length=max_length,truncation=True,is_split_into_words=True)
    input_ids = converter["input_ids"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids = input_ids, attention_mask = converter['attention_mask'].to(device))

    probs = F.softmax(outputs.logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)
    predlist = preds.tolist()
    
    i = 0
    word = []

    input_list = input_ids.tolist()[0]

    for k in range(len(predlist[0])):
        if predlist[0][k] == 1:
            word.append(input_list[k])

    regspace = r'\s'

    spaces = re.finditer(regspace,desc)
    descNoSpaces = desc.replace(" ","")
    spaceInd = []

    for match in spaces:
        spaceInd.append(match.span()[0])

    word = ""
    last = ""
    while i < len(converter.word_ids()):
        
        if predlist[0][i] == 1:
            if converter.word_ids()[i] != last and converter.word_ids()[i] != None:
                word = word + re.findall(regtest,desc)[converter.word_ids()[i]]
            
            last = converter.word_ids()[i]
    
        i +=1

    

    if len(word) > 0:
        found = descNoSpaces.find(word)
        if found != -1:
            temp = spaceInd
            for t in temp:
                if t < found:
                    del spaceInd[0]
            for s in spaceInd:
                if s > len(word):
                    break;
                word = word[:s-found] + " " + word[s-found:]
            
            if word[-1] == " ":
                word = word[:-1]
            extractlist.append({"id":editionV.replace(".",str(v)),"headword":word,"description":desc})
            v += 1
    
    
        

with open ("TREDJE_EXTRACTED.json","w",encoding = "utf-8") as f:
    json.dump(extractlist,f,indent = 4,ensure_ascii = False)

100%|██████████| 249550/249550 [56:06<00:00, 74.12it/s] 


In [19]:
print(len(extractlist))

124158
